## Use Case Focus

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import numpy as np
from datetime import datetime
import itertools
import os

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Constants and model list
MODEL_NAMES = [
    "albert/albert-base-v2",
    "albert/albert-large-v2",
    "microsoft/deberta-v3-large",
    # "microsoft/deberta-v2-xlarge",
    "FacebookAI/roberta-large",
    "google-bert/bert-base-uncased",
    "ProsusAI/finbert",
    "PHILIPPUNI/distilbert-amazon-software-reviews-finetuned",
    "justinlamlamlam/softwareengineering",
    "answerdotai/ModernBERT-large",
    "milyiyo/distilbert-base-uncased-finetuned-amazon-review"
]
LABEL_COLUMN = "label"

# Hyperparameter grid
EPOCHS_LIST = [1, 3, 4, 9, 12]
BATCH_SIZES = [8]
N_SPLITS_LIST = [5, 10]

# Load data
data = pd.read_csv("type_val_single.csv")
label_encoder = LabelEncoder()
data[LABEL_COLUMN] = label_encoder.fit_transform(data[LABEL_COLUMN])
texts = data["sentence"].tolist()
labels = data[LABEL_COLUMN].tolist()

# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Training loop
def train_model(model, train_loader, optimizer, criterion, epochs, fold):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}")
        for batch in progress_bar:
            optimizer.zero_grad()
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=total_loss / (progress_bar.n + 1))

# Evaluation
def evaluate_model(model, val_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            preds = model(**inputs).logits.argmax(dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

# Generate folds once and reuse them
def generate_kfold_splits(texts, labels, n_splits):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    return list(skf.split(texts, labels))

# Save best result to file
def save_best_result(log_path, header, result, best_combo_summary):
    with open(log_path, "w") as f:
        f.write(header + "\n\n")
        f.write(result + "\n")
        f.write("\n===== BEST COMBINATION =====\n")
        f.write(best_combo_summary + "\n")

# Begin experiment
for n_splits in N_SPLITS_LIST:
    folds = generate_kfold_splits(texts, labels, n_splits)

    for model_name in MODEL_NAMES:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        safe_model_name = model_name.replace('/', '-')
        log_file = f"{safe_model_name}_{LABEL_COLUMN}.txt"

        best_f1 = 0
        best_result = ""
        best_combo = ""

        for epochs, batch_size in itertools.product(EPOCHS_LIST, BATCH_SIZES):
            print(f"\nModel: {model_name} | EPOCHS={epochs}, BATCH_SIZE={batch_size}, K-FOLD={n_splits}")
            all_accuracies, all_precisions, all_recalls, all_f1s = [], [], [], []

            for fold, (train_idx, val_idx) in enumerate(folds):
                train_texts = [texts[i] for i in train_idx]
                val_texts = [texts[i] for i in val_idx]
                train_labels = [labels[i] for i in train_idx]
                val_labels = [labels[i] for i in val_idx]

                train_dataset = TextDataset(train_texts, train_labels, tokenizer)
                val_dataset = TextDataset(val_texts, val_labels, tokenizer)

                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                model = AutoModelForSequenceClassification.from_pretrained(
                    model_name,
                    num_labels=len(set(labels)),
                    ignore_mismatched_sizes=True
                ).to(device)

                optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
                criterion = torch.nn.CrossEntropyLoss()

                train_model(model, train_loader, optimizer, criterion, epochs, fold)
                y_true, y_pred = evaluate_model(model, val_loader)

                acc = accuracy_score(y_true, y_pred)
                prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

                all_accuracies.append(acc)
                all_precisions.append(prec)
                all_recalls.append(rec)
                all_f1s.append(f1)

            avg_accuracy = np.mean(all_accuracies)
            avg_precision = np.mean(all_precisions)
            avg_recall = np.mean(all_recalls)
            avg_f1 = np.mean(all_f1s)

            if avg_f1 > best_f1:
                best_f1 = avg_f1
                best_combo = f"Model: {model_name}\nLabel Column: {LABEL_COLUMN}\nEpochs: {epochs}, Batch Size: {batch_size}, K-Fold: {n_splits}"
                best_result = (
                    f"Accuracy: {avg_accuracy:.4f}\n"
                    f"Precision: {avg_precision:.4f}\n"
                    f"Recall: {avg_recall:.4f}\n"
                    f"F1-Score: {avg_f1:.4f}"
                )

        # Save best result to file
        header = f"Best Hyperparameter Combination for {model_name} on {LABEL_COLUMN}"
        save_best_result(log_file, header, best_result, best_combo)
        print(f"\nBest combination saved to {log_file}")



Model: albert/albert-base-v2 | EPOCHS=1, BATCH_SIZE=8, K-FOLD=5


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fold 1 Epoch 1: 100%|██████████| 15/15 [00:02<00:00,  7.45it/s, loss=1.15]
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: albert/albert-base-v2 | EPOCHS=3, BATCH_SIZE=8, K-FOLD=5


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fold 1 Epoch 3: 100%|██████████| 15/15 [00:01<00:00,  9.03it/s, loss=0.817]
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model: albert/albert-base-v2 | EPOCHS=4, BATCH_SIZE=8, K-FOLD=5


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at milyiyo/distilbert-base-uncased-finetuned-amazon-review and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fold 1 Epoch 9: 100%|██████████| 17/17 [00:00<00:00, 17.18it/s, loss=0.166]
/usr/local/lib/python3.11/site-packages/sklearn/metr


Model: milyiyo/distilbert-base-uncased-finetuned-amazon-review | EPOCHS=12, BATCH_SIZE=8, K-FOLD=10


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at milyiyo/distilbert-base-uncased-finetuned-amazon-review and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fold 1 Epoch 12: 100%|██████████| 17/17 [00:00<00:00, 17.09it/s, loss=0.0849]
/usr/local/lib/python3.11/site-packages/sklearn/me


Best combination saved to milyiyo-distilbert-base-uncased-finetuned-amazon-review_label.txt


/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Structure Focus

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import numpy as np
from datetime import datetime
import itertools
import os

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Constants and model list
MODEL_NAMES = [
    # "albert/albert-base-v2",
    # "albert/albert-large-v2",
    # "microsoft/deberta-v3-large",
    # "microsoft/deberta-v2-xlarge",
    # "FacebookAI/roberta-large",
    # "google-bert/bert-base-uncased",
    # "ProsusAI/finbert",
    # "PHILIPPUNI/distilbert-amazon-software-reviews-finetuned",
    # "justinlamlamlam/softwareengineering",
    # "answerdotai/ModernBERT-large",
    "milyiyo/distilbert-base-uncased-finetuned-amazon-review"
]
LABEL_COLUMN = "structure_focus"

# Hyperparameter grid
EPOCHS_LIST = [1, 3, 4, 9, 12]
BATCH_SIZES = [8]
N_SPLITS_LIST = [5, 10]

# Load data
data = pd.read_csv("type_classification-validation.csv")
label_encoder = LabelEncoder()
data[LABEL_COLUMN] = label_encoder.fit_transform(data[LABEL_COLUMN])
texts = data["sentence"].tolist()
labels = data[LABEL_COLUMN].tolist()

# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Training loop
def train_model(model, train_loader, optimizer, criterion, epochs, fold):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}")
        for batch in progress_bar:
            optimizer.zero_grad()
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=total_loss / (progress_bar.n + 1))

# Evaluation
def evaluate_model(model, val_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            preds = model(**inputs).logits.argmax(dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

# Generate folds once and reuse them
def generate_kfold_splits(texts, labels, n_splits):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    return list(skf.split(texts, labels))

# Save best result to file
def save_best_result(log_path, header, result, best_combo_summary):
    with open(log_path, "w") as f:
        f.write(header + "\n\n")
        f.write(result + "\n")
        f.write("\n===== BEST COMBINATION =====\n")
        f.write(best_combo_summary + "\n")

# Begin experiment
for n_splits in N_SPLITS_LIST:
    folds = generate_kfold_splits(texts, labels, n_splits)

    for model_name in MODEL_NAMES:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        safe_model_name = model_name.replace('/', '-')
        log_file = f"{safe_model_name}_{LABEL_COLUMN}.txt"

        best_f1 = 0
        best_result = ""
        best_combo = ""

        for epochs, batch_size in itertools.product(EPOCHS_LIST, BATCH_SIZES):
            print(f"\nModel: {model_name} | EPOCHS={epochs}, BATCH_SIZE={batch_size}, K-FOLD={n_splits}")
            all_accuracies, all_precisions, all_recalls, all_f1s = [], [], [], []

            for fold, (train_idx, val_idx) in enumerate(folds):
                train_texts = [texts[i] for i in train_idx]
                val_texts = [texts[i] for i in val_idx]
                train_labels = [labels[i] for i in train_idx]
                val_labels = [labels[i] for i in val_idx]

                train_dataset = TextDataset(train_texts, train_labels, tokenizer)
                val_dataset = TextDataset(val_texts, val_labels, tokenizer)

                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                model = AutoModelForSequenceClassification.from_pretrained(
                    model_name,
                    num_labels=len(set(labels)),
                    ignore_mismatched_sizes=True
                ).to(device)

                optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
                criterion = torch.nn.CrossEntropyLoss()

                train_model(model, train_loader, optimizer, criterion, epochs, fold)
                y_true, y_pred = evaluate_model(model, val_loader)

                acc = accuracy_score(y_true, y_pred)
                prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

                all_accuracies.append(acc)
                all_precisions.append(prec)
                all_recalls.append(rec)
                all_f1s.append(f1)

            avg_accuracy = np.mean(all_accuracies)
            avg_precision = np.mean(all_precisions)
            avg_recall = np.mean(all_recalls)
            avg_f1 = np.mean(all_f1s)

            if avg_f1 > best_f1:
                best_f1 = avg_f1
                best_combo = f"Model: {model_name}\nLabel Column: {LABEL_COLUMN}\nEpochs: {epochs}, Batch Size: {batch_size}, K-Fold: {n_splits}"
                best_result = (
                    f"Accuracy: {avg_accuracy:.4f}\n"
                    f"Precision: {avg_precision:.4f}\n"
                    f"Recall: {avg_recall:.4f}\n"
                    f"F1-Score: {avg_f1:.4f}"
                )

        # Save best result to file
        header = f"Best Hyperparameter Combination for {model_name} on {LABEL_COLUMN}"
        save_best_result(log_file, header, best_result, best_combo)
        print(f"\nBest combination saved to {log_file}")


In [ ]:
print("finish")